In [6]:
pip install xlrd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd

# Load the Excel file
excel_file_path = '/Users/akankshasathish/Desktop/scoreboard_link.xls'  # Update this to your file's path
df = pd.read_excel(excel_file_path)

# Rename the columns
df = df.rename(columns={'URL': 'Match_link', 'Data': 'Match_data'})

# Save as a CSV file
csv_file_path = '/Users/akankshasathish/Desktop/match_schedule.csv'  # Update this to your desired path
df.to_csv(csv_file_path, index=False)

In [9]:
display(df)

,ID,Match_link,Match_data
0,0,https://clublocker.com/leagues/match/184188,"Thu, Jul 20, 23 @6:00pm"
1,1,https://clublocker.com/leagues/match/184187,"Thu, Jul 20, 23 @6:00pm"
2,2,https://clublocker.com/leagues/match/184160,"Thu, Jul 13, 23 @6:00pm"
3,3,https://clublocker.com/leagues/match/184124,"Thu, Jul 6, 23 @6:00pm"
4,4,https://clublocker.com/leagues/match/184122,"Thu, Jun 29, 23 @6:00pm"
...,...,...,...
145,145,https://clublocker.com/leagues/match/156470,"Wed, Nov 10, 21 @6:00am"
146,146,https://clublocker.com/leagues/match/156471,"Wed, Nov 10, 21 @6:00am"
147,147,https://clublocker.com/leagues/match/156472,"Wed, Nov 10, 21 @6:00am"
148,148,https://clublocker.com/leagues/match/156464,"Wed, Oct 27, 21 @6:00am"


In [10]:
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

urls_csv_path = '/Users/akankshasathish/Desktop/scoreboard_ids.csv'
urls_df = pd.read_csv(urls_csv_path)

safari_options = Options()

driver = webdriver.Safari(options=safari_options)

matches_data = []

for index, row in urls_df.iterrows():
    url = row['match_links']
    match_date = row['match_date']
    match_id = row['match_id']

    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    match_div = soup.find_all('div', class_='match')
    for i, div in enumerate(match_div):
        player_names_span = div.find_all('span', class_='ellipsis')
        players_a = player_names_span[0].get_text(strip=True).split('|')
        players_b = player_names_span[1].get_text(strip=True).split('|')

        scores_span = div.find_all('span', {'_ngcontent-fpu-c246': ''})
        filtered_scores_span = [span for span in scores_span if not span.has_attr('class')]
        scores = filtered_scores_span[0].get_text(strip=True).split('-')

        if len(scores) == 1:
            scores = ["Match Disqualified", "Match Disqualified"]

        matches_data.append({
            'Player A1' : players_a[0],
            'Player A2' : players_a[1],
            'Score Team A' : scores[0],
            'Player B1' : players_b[0],
            'Player B2' : players_b[1],
            'Score Team B' : scores[1],
            'Match Date' : match_date,
            'Match ID' : match_id


        })

driver.quit()

df_matches = pd.DataFrame(matches_data)

if not df_matches.empty:
    excel_path = '/Users/akankshasathish/Desktop/squash_scores_date.xlsx'
    df_matches.to_excel(excel_path, index=False)
    print(f"Data scraped and saved to Excel file at {excel_path}")
else:
    print("No data was found to scrape.")



                


Data scraped and saved to Excel file at /Users/akankshasathish/Desktop/squash_scores_date.xlsx


In [11]:
display(df_matches)

,Player A1,Player A2,Score Team A,Player B1,Player B2,Score Team B,Match Date,Match ID
0,Hugh Upjohn,Paul Deziel,0,Fungpiew Lim,Joanne Ditommaso,3,"Thu, Jul 20, 23",184188
1,Sharisse Ying-Ho Lin,Shelley Neil,0,Wayne Maunder,Matthew Waite,3,"Thu, Jul 20, 23",184188
2,Zhelin Bu,Ernie Chan,0,Ian Binstead,Lam Uong,3,"Thu, Jul 20, 23",184187
3,Not on Roster,Guy Steuart,0,Luis Moran,Janine Waite,3,"Thu, Jul 20, 23",184187
4,Chad Forbes,Liz Tuck,2,Fungpiew Lim,Matthew Waite,3,"Thu, Jul 13, 23",184160
...,...,...,...,...,...,...,...,...
563,Brian Maclean,Warren Woodman,1,Michael Birch,Mikhail Din,3,"Wed, Oct 27, 21",156464
564,Christian Schwendener,Gary Bombay,1,Kevin McElgunn,Greg Coutts,3,"Wed, Oct 27, 21",156464
565,Shawn Patton,Gregg Ashby,3,Bruce Matthews,Rod Nicolls,1,"Wed, Oct 27, 21",156463
566,Ricardo Sanchez,Randy Mottus,3,Simon Whyte,James Woodcock,0,"Wed, Oct 27, 21",156463
